# Setting up an OpenMM iMD simulation

This notebook demonstrates how to set up an OpenMM simulation for use with NanoVer from scratch.
We take AMBER files for neuraminidase with oseltamivir (AKA tamiflu) bound, create an OpenMM system and 
set it up with NanoVer.

## Set up the OpenMM Simulation

We start by creating an OpenMM simulation for the neuraminidase-oseltamivir system using the relevant pre-prepared AMBER files. OpenMM also supports Gromacs and CHARMM files, and can be customized for many other uses. 

In [1]:
import openmm as mm
import openmm.unit as unit 
import openmm.app as app

In [2]:
prmtop = app.AmberPrmtopFile("openmm_files/3TI6_ose_wt.top")
amber_coords = app.AmberInpcrdFile("openmm_files/3TI6_ose_wt.rst")

Let's keep the simulation simple by using implicit solvent and no constraints.

In [3]:
system = prmtop.createSystem(nonbondedMethod=app.CutoffPeriodic, 
                             nonbondedCutoff=2*unit.nanometer, 
                             implicitSolvent=app.OBC2,
                             constraints=None)

/opt/homebrew/Caskroom/miniforge/base/envs/nanover-dev/lib/python3.12/site-packages/openmm/app/internal/amber_file_parser.py:1168: UserWarning: Non-optimal GB parameters detected for GB model OBC2
  warnings.warn(


Now let's define the integrator we will use to generate the dynamics. In this case, we use OpenMM's Langevin integrator, defining the temperature, friction coefficient and simulation time step.

In [4]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.001*unit.picoseconds)

Now we create an OpenMM simulation from the topology, system and integrator that we have defined, and define the positions using the AMBER input coordinate file.

In [5]:
simulation = app.Simulation(prmtop.topology, system, integrator)

In [6]:
simulation.context.setPositions(amber_coords.positions)
if amber_coords.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*amber_coords.boxVectors)

Let's minimize the energy to create a stable conformation that can be used to run dynamics:

In [7]:
# NBVAL_SKIP
simulation.minimizeEnergy()

Now let's run a few steps to check it's stable, printing the potential energy and temperature every 500 steps.

In [8]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)

In [9]:
import sys

In [10]:
# NBVAL_SKIP
simulation.reporters.append(app.StateDataReporter(sys.stdout, 500, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(5000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,-35883.364876014224,206.54079050509745
1000,-33071.34326956891,243.65622917125617
1500,-31584.641914588443,268.74152673118033
2000,-30259.581215125552,278.7086006274739
2500,-29808.444130164615,285.64465667112916
3000,-29605.372993690005,289.4358602003129
3500,-29430.563789588443,294.73696042840896
4000,-29160.257423621646,296.02520556113
4500,-29044.606392127505,297.63231302721914
5000,-28838.400825721255,301.4710805902629


Looks good! Now, let's remove that reporter and set it up for use with NanoVer.

In [11]:
simulation.reporters.clear()

The following cell outputs the simulation as a NanoVer OpenMM XML, which contains the PDB with the topology and the OpenMM XML (containing the System and Integrator). This lets you take the simulation we've set up here and use it directly with NanoVer without having to repeat this procedure each time, perfect if you just want to run a simulation quickly. Here's how you would run this simulation from the file created using the terminal: 

```bash 
$ nanover-omni --omm neuraminidase_nanover.xml
```

In [12]:
from nanover.openmm.serializer import serialize_simulation

with open('neuraminidase_nanover.xml','w') as f:
    f.write(serialize_simulation(simulation))

In the following section, we'll set up the simulation by hand, essentially replicating here in our notebook what `nanover-omni --omm` does when running NanoVer via the terminal.

## Setting up the OpenMM simulation with NanoVer

Now that we've created an OpenMM simulation, let's serve it with NanoVer to run an interactive molecular dynamics simulation. We do so using by passing the simulation to the `OmniRunner` class as an `OpenMMSimulation`, just as we did in the [nanotube notebook](./openmm_nanotube.ipynb).

First, we need to import the relevant classes:

In [13]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

Then, we create an instance of the `OpenMMSimulation` class for the neuraminidase simulation defined above. Let's create the instance directly from the simulation we've set up in this notebook.

In [14]:
neuraminidase_simulation = OpenMMSimulation.from_simulation(simulation)

Lastly, create an instance of the `OmniRunner` class and run the simulation.

In [15]:
imd_runner = OmniRunner.with_basic_server(neuraminidase_simulation, name='neuraminidase-omm-server')
imd_runner.next()

Great! Now we have our simulation up and running! Connect to it from VR and you'll see something like this:

![NanoVer neuraminidase](./images/neuraminidase_ball_and_stick.png)

Let's leave it running in the background and turn our attention to an important aspect of molecular visualisation: making things look pretty!

## Let's make it pretty!

Ball and stick is so 2001, let's make it look cool. We'll also make it so that if you interact with any of the atoms of oseltamivir, you'll interact with the entire molecule as a group, which is more stable.

First, we connect a client so we can modify the shared state.

In [16]:
from nanover.app import NanoverImdClient
client = NanoverImdClient.connect_to_single_server(port=imd_runner.app_server.port)
client.subscribe_to_frames()
client.wait_until_first_frame();

We define a couple of handy methods for playing with selections and colour gradients.

In [17]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.colormaps[name]
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

In [18]:
from nanover.mdanalysis import frame_data_to_mdanalysis
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

Hide the 'root' selection, it's getting in the way of our creativity.

In [19]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

Let's select the protein.

In [20]:
protein = client.create_selection("Protein", [])

In [21]:
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

We'll colour it and render with a spline, or ribbon, renderer.
Some things you can try: 
* Change the render: `spline`, `geometric spline`. Or comment out the `sequence` line and try `liquorice`,`noodles`, `cycles`, `ball and stick`.
* Change the color: set it to be one color, or try some different matplotlib [color maps](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html), e.g. `rainbow` or `magma`.
* Change the scale.

In [22]:
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('rainbow')
            },
            'render': 'geometric spline',
            'scale': 0.2
        }
    protein.interaction_method = 'single'

Let's reintroduce the ligand, oseltamivir, and make it so we interact with it as a group.

In [23]:
# Select ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [24]:
with ligand.modify():
    ligand.renderer = {
            'color': 'cpk',
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligand.velocity_reset = True
    ligand.interaction_method = 'group'

If you've done all that, you'll have something that looks like this:

![Neuraminidase Geometric](./images/neuraminidase_geometric_spline.png)

## Tidying Up After Yourself

In [25]:
client.close()

In [26]:
imd_runner.close()

## Next Steps